In [44]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import torch.optim as optim

In [45]:
torch.manual_seed(42)

In [46]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [47]:
df = pd.read_csv('/content/fashion-mnist_train.csv')
df

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,1,0,0,0,0,0,0,0,0,0,...,73,0,0,0,0,0,0,0,0,0
59997,8,0,0,0,0,0,0,0,0,0,...,160,162,163,135,94,0,0,0,0,0
59998,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
x = df.iloc[:,1:].values
y = df.iloc[:,0].values

In [50]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [51]:
x_train = x_train/255.0
x_test = x_test/255.0

In [52]:
class customDataset(Dataset):
  def __init__(self,features,labels):
    self.features = torch.tensor(features, dtype=torch.float32)
    self.labels = torch.tensor(labels, dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self,idx):
    x = self.features[idx]
    y = self.labels[idx]
    return x,y

In [53]:
train_loader = DataLoader(customDataset(x_train,y_train),batch_size=32,shuffle=True, pin_memory=True)
test_loader = DataLoader(customDataset(x_test,y_test),batch_size=32,shuffle=False,  pin_memory=True)

In [62]:
class MyNetwork(nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(num_features,128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(p=0.3),
        nn.Linear(128,64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Dropout(p=0.3),
        nn.Linear(64,10))

  def forward(self,x):
    x = self.model(x)
    return x

In [63]:
model = MyNetwork(x_train.shape[1])
model = model.to(device)

optimizer = optim.SGD(model.parameters(),lr=0.1, weight_decay=1e-4)

loss_function = nn.CrossEntropyLoss()

In [64]:
for epochs in range(100):
  total_epoch_loss=0

  for x,y in train_loader:
    x = x.to(device)
    y = y.to(device)

    output = model(x)

    loss = loss_function(output,y)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()
    total_epoch_loss = total_epoch_loss + loss.item()

  print(f"Epoch: {epochs+1}, Loss: {total_epoch_loss/len(train_loader)}")

Epoch: 1, Loss: 0.6256776882211367
Epoch: 2, Loss: 0.4982376752992471
Epoch: 3, Loss: 0.46010538974404336
Epoch: 4, Loss: 0.4353683769305547
Epoch: 5, Loss: 0.419320604339242
Epoch: 6, Loss: 0.4041203481008609
Epoch: 7, Loss: 0.3905387718329827
Epoch: 8, Loss: 0.3802584114074707
Epoch: 9, Loss: 0.3777289008249839
Epoch: 10, Loss: 0.3700699914842844
Epoch: 11, Loss: 0.367465089738369
Epoch: 12, Loss: 0.3580042658696572
Epoch: 13, Loss: 0.3544128436297178
Epoch: 14, Loss: 0.346828492179513
Epoch: 15, Loss: 0.34716397122045356
Epoch: 16, Loss: 0.3414345298310121
Epoch: 17, Loss: 0.33740889925261336
Epoch: 18, Loss: 0.332828682889541
Epoch: 19, Loss: 0.32705044343322515
Epoch: 20, Loss: 0.3269968666980664
Epoch: 21, Loss: 0.32775757918755216
Epoch: 22, Loss: 0.3247317676718036
Epoch: 23, Loss: 0.31912228437761464
Epoch: 24, Loss: 0.31953005809833607
Epoch: 25, Loss: 0.3137063124701381
Epoch: 26, Loss: 0.31012241668750845
Epoch: 27, Loss: 0.30889108665039144
Epoch: 28, Loss: 0.3077559935996

In [65]:
model.eval()

MyNetwork(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [66]:
total = 0
correct = 0

with torch.no_grad():
  for x,y in test_loader:
    x = x.to(device)
    y = y.to(device)

    # Convert input tensor to float
    x = x.float()

    outputs = model(x)

    _,predicted = torch.max(outputs,1)

    total += y.size(0)
    correct += (predicted == y).sum().item()

print(f"Accuracy: {correct/total}")

Accuracy: 0.8899166666666667


In [67]:
total = 0
correct = 0

with torch.no_grad():
  for x,y in train_loader:
    x = x.to(device)
    y = y.to(device)

    # Convert input tensor to float
    x = x.float()

    outputs = model(x)

    _,predicted = torch.max(outputs,1)

    total += y.size(0)
    correct += (predicted == y).sum().item()

print(f"Accuracy: {correct/total}")

Accuracy: 0.9419166666666666
